## snpgenie testing notebook

Updated files can be found here: https://github.com/dmnfarrell/gordon-group/tree/master/mbovis_annotation

links:

* https://github.com/JosephCrispell/GeneralTools/tree/master/ProcessingPipeline
* [Woodchester Park eLife paper](https://elifesciences.org/articles/45833)
* [An African origin for Mycobacterium bovis](https://academic.oup.com/emph/article/2020/1/49/5719036)
* http://www.htslib.org/doc/bcftools.html
* https://mtbgenomicsworkshop.readthedocs.io/
* [BCFtools cheat sheet](https://gist.github.com/elowy01/93922762e131d7abd3c7e8e166a74a0b)
* https://biopython.org/wiki/Category:Cookbook
* https://pyvcf.readthedocs.io/en/v0.4.6/INTRO.html
* https://gatkforums.broadinstitute.org/gatk/discussion/1268/what-is-a-vcf-and-how-should-i-interpret-it

In [4]:
import sys,os,shutil,subprocess
import random
import glob, time
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from gzip import open as gzopen
sys.path.append('pathogenie')
from snpgenie import tools, aligners, app, trees, plotting

## Mbovis samples from SRA

In [4]:
sra = pd.read_csv('../sra_mbovis_runtable.csv')

## Get master table with Loiseau et al. meta data¶

In [3]:
df = pd.read_excel('../Loiseau_TableS1.xlsx')
master = pd.merge(df,sra,left_on='BIOSAMPLE',right_on='BioSample',how='inner')
master.to_csv('../mbovis_sra_master.csv',index=False)
cols = ['Run','CLONAL_COMPLEX','COUNTRY_ISOLATION','LibraryLayout','Host','ReleaseDate','Bytes','Bases','GENOME_COVERAGE']
#master.columns
#master.geo_loc_name_country.value_counts()

## Representative samples of lineages

In [4]:
sub=master[(master.LibraryLayout=='PAIRED') & (master.COUNTRY_ISOLATION.notnull()) & (master.CLONAL_COMPLEX=='Eu2')]
#print (sub[cols])
#sub[cols].to_csv('../temp.csv')
accessions=['SRR1792002','SRR5216872','SRR5486071','ERR841808','ERR125601',
            'SRR8063654','SRR8063665','SRR8065079','SRR1791768','SRR1791960','ERR2815558']
testset = master[master.Run.isin(accessions)]
print (testset[cols])

             Run CLONAL_COMPLEX COUNTRY_ISOLATION LibraryLayout        Host           ReleaseDate        Bytes        Bases  GENOME_COVERAGE
36     ERR841808            Eu1                UK        PAIRED         Cow  2015-12-02T00:00:00Z  130651179.0  196040478.0            39.78
245   SRR5216872            Eu1       New Zealand        PAIRED      BOVINE  2017-01-31T00:00:00Z  194523112.0  295803584.0            41.39
464   SRR5486071       Unknown2          Ethiopia        PAIRED  Bos taurus  2017-04-28T00:00:00Z  251875435.0  445066100.0            50.71
677    ERR125601            Eu1           Ireland        PAIRED         NaN  2012-10-01T00:00:00Z  277182504.0  346340680.0            68.83
765   ERR2815558            Af1               NaN        PAIRED         NaN  2018-09-29T00:00:00Z  162711425.0  367702705.0            67.64
886   SRR1791768            Eu2            Canada        PAIRED      Cattle  2015-03-11T00:00:00Z  340596851.0  450406275.0            37.69
1076  SRR1791

In [ ]:
# fetch from SRA
def fetch_sra_reads(df,path):
    """download a subset of reads """
    
    for i,r in df.iterrows():
        files = glob.glob(os.path.join(path,r.Run+'*'))        
        if len(files)==0:
            cmd = 'fastq-dump --split-files {n} --outdir {o}'.format(n=r.Run,o=path)
            print (cmd)
            subprocess.check_output(cmd,shell=True)

fetch_sra_reads(testset,path='/storage/btbgenie/mbovis_sra')

## Blasting to known contaminants

* https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-2684-x#Sec2

In [ ]:
#get unmapped reads from a bam file

infile='mapped/3_S57_L001.bam'
cmd = 'samtools view -b -f 4 {i} > unmapped.bam'.format(i=infile)
print (cmd)
#align with bwa to known index of contaminants


## Examine FASTQ file quality and define trimming parameters

In [ ]:
reload(tools)

df = tools.fastq_to_dataframe(testfile)
df.length.hist()

In [ ]:
def trim_reads_default(filename, outfile, right_quality=30):
    """Trim adapters"""
        
    fastq_parser = SeqIO.parse(gzopen(filename, "rt"), "fastq")
    c=0
    out = gzopen(outfile, "wt")
    for record in fastq_parser:
        score = record.letter_annotations["phred_quality"]         
        for i in range(len(score)-1,0,-1):
            if score[i] >= right_quality:
                break
        #trimmed.append(record[:i])
        #print (record[:i])        
        #c+=1
        #if c>100:
        #    break    
        SeqIO.write(record[:i],out,'fastq')
    return

testfile = 'mbovis_sra/SRR1791711_1.fastq.gz'
st=time.time()
trim_reads_default(testfile,'trimmed.fastq.gz')
print (time.time()-st)
df = tools.fastq_to_dataframe('trimmed.fastq.gz')
df.length.hist()

## multiprocess trimming

In [ ]:
def split_fastq(filename, chunks=4):
    """Split fastq file"""
    
    names=[]
    tmp = '/tmp'
    tmpfiles = []
    record_iter = SeqIO.parse(gzopen(filename, "rt"), "fastq")
    length = 
    for i, batch in enumerate(tools.batch_iterator(record_iter, chunksize)):
        tempfile = "group_%i.fastq" % (i + 1)
        tmpfiles.append(tempfile)
        names.append(tempfile)
        with open(tempfile, "w") as handle:
            count = SeqIO.write(batch, handle, "fastq")
        print("Wrote %i records to %s" % (count, tempfile))
    return names

split_fastq(testfile)

In [ ]:
tools.plot_fastq_qualities(testfile,limit=100000)
plt.savefig('fastq_quals.png',dpi=100)
tools.plot_fastq_gc_content(testfile, limit=50000)
plt.savefig('gc.png',dpi=100)

## Get sample ids from fastq files

In [ ]:
path1='/storage/btbgenie/albania'
files = glob.glob(os.path.join(path1,'*.fastq.gz'))
path2='/storage/btbgenie/mbovis_sra'
files = glob.glob(os.path.join(path2,'*.fastq.gz'))
#add outgroup file
#files.append
files = app.get_files_from_paths([path1,path2])

df = app.get_samples(files)
df

## Align the FASTQ files against reference and create VCF files

In [ ]:
reload(aligners)
ref = app.mbovis_genome
samples = app.get_samples(files)
app.align_reads(samples, idx=ref, threads=10, overwrite=False)

## remove duplicated reads

In [ ]:
cmd = 'samtools rmdup A_reads.bt2.sorted.bam A_reads.bt2.sorted.noDups.bam'

## Keeping unmapped reads

https://gist.github.com/davfre/8596159

## variant calling

In [119]:
reload(app)
ref = app.mbovis_genome
bam_files = glob.glob('mapped/*.bam')
st=time.time()
vcf_file = app.variant_calling(bam_files, ref, 'mapped', threads=10, overwrite=True)
print (time.time()-st)

[      1  434991  869981 1304971 1739962 2174952 2609942 3044933 3479923
 3914913 4349904]
parallel bcftools mpileup -r {1} -a "AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR" -O b  --min-MQ 60 -o {2} -f /home/damien/.config/snpgenie/genome/Mbovis_AF212297.fa mapped/ERR551704.bam mapped/6_S7_L001.bam mapped/3_S57_L001.bam ::: "LT708304.1":1-434990 "LT708304.1":434991-869980 "LT708304.1":869981-1304970 "LT708304.1":1304971-1739961 "LT708304.1":1739962-2174951 "LT708304.1":2174952-2609941 "LT708304.1":2609942-3044932 "LT708304.1":3044933-3479922 "LT708304.1":3479923-3914912 "LT708304.1":3914913-4349903 :::+ /tmp/1.bcf /tmp/434991.bcf /tmp/869981.bcf /tmp/1304971.bcf /tmp/1739962.bcf /tmp/2174952.bcf /tmp/2609942.bcf /tmp/3044933.bcf /tmp/3479923.bcf /tmp/3914913.bcf
bcftools concat /tmp/1.bcf /tmp/434991.bcf /tmp/869981.bcf /tmp/1304971.bcf /tmp/1739962.bcf /tmp/2174952.bcf /tmp/2609942.bcf /tmp/3044933.bcf /tmp/3479923.bcf /tmp/3914913.bcf -O b -o mapped/raw.bcf
bcftools call -V indels --pl

## freebayes calling

In [204]:
fbcmd = 'freebayes'

ref=app.mbovis_genome
bam_files = glob.glob('mapped/*.bam')
print (bam_files)
out='mapped/freebayes.vcf'
cmd = '{fb} -p 2 -P 0 -C 2 -F 0.05 --min-coverage 10'\
         ' --min-repeat-entropy 1.0 -q 40 -m 20 --strict-vcf'\
        ' -f {r} -L files.txt > {o}'.format(r=ref,o=out,fb=fbcmd)        
print (cmd)

['mapped/ERR551704.bam', 'mapped/6_S7_L001.bam', 'mapped/3_S57_L001.bam']
freebayes -p 2 -P 0 -C 2 -F 0.05 --min-coverage 10 --min-repeat-entropy 1.0 -q 40 -m 20 --strict-vcf -f /home/damien/.config/snpgenie/genome/Mbovis_AF212297.fa -L files.txt > mapped/freebayes.vcf


In [ ]:
v1 = vcf_to_dataframe('result.vcf')
print (v1.var_type.value_counts())

In [ ]:
v2 = vcf_to_dataframe('mapped/raw.bcf')
print (v2.var_type.value_counts())

## filtering

```
FORMAT/AD   .. Allelic depth (Number=R,Type=Integer)
FORMAT/ADF  .. Allelic depths on the forward strand (Number=R,Type=Integer)
FORMAT/ADR  .. Allelic depths on the reverse strand (Number=R,Type=Integer)
FORMAT/DP   .. Number of high-quality bases (Number=1,Type=Integer)
FORMAT/SP   .. Phred-scaled strand bias P-value (Number=1,Type=Integer)
FORMAT/SCR  .. Number of soft-clipped reads (Number=1,Type=Integer)

INFO/AD     .. Total allelic depth (Number=R,Type=Integer)
INFO/ADF    .. Total allelic depths on the forward strand (Number=R,Type=Integer)
INFO/ADR    .. Total allelic depths on the reverse strand (Number=R,Type=Integer)
INFO/SCR    .. Number of soft-clipped reads (Number=1,Type=Integer)

FORMAT/DV   .. Deprecated in favor of FORMAT/AD; Number of high-quality non-reference bases, (Number=1,Type=Integer)
FORMAT/DP4  .. Deprecated in favor of FORMAT/ADF and FORMAT/ADR; Number of high-quality ref-forward, ref-reverse,
               alt-forward and alt-reverse bases (Number=4,Type=Integer)
FORMAT/DPR  .. Deprecated in favor of FORMAT/AD; Number of high-quality bases for each observed allele (Number=R,Type=Integer)
INFO/DPR    .. Deprecated in favor of INFO/AD; Number of high-quality bases for each observed allele (Number=R,Type=Integer)
```

Joe filters: DP - 30, DP4 - 4, MQ - 35, SUP - 0.95 and COV - 0.05. GQ and FQ = 0

## Other options

```
biallelic: -m2 -M2
uninformative sites, where the minor allele is present in at least one individual: --min-ac 1:minor
```

In [ ]:
import vcf
vcf_reader = vcf.Reader(open('mapped/filtered.vcf.gz', 'rb')) 
for record in vcf_reader:
    print (record)
    for sample in record.samples:
        print (sample)

In [ ]:
cmd = 'bcftools filter -e "QUAL<=40 && DP>=30 && DP4>=4 && MQ>=35" -o filtered.vcf.gz -O z mapped/calls.vcf'
print (cmd)
tmp = subprocess.check_output(cmd,shell=True)
v = vcf_to_dataframe('filtered.vcf.gz')
v.var_type.value_counts()

## consequence calling for vcf

In [94]:
reload(app)
ref=app.mbovis_genome
gff = '../snpgenie/data/Mbovis_csq_format.gff'
out = 'mapped/csq.tsv'
def csq_call():
    cmd = 'bcftools csq -f {r} -g {g} mapped/filtered.vcf.gz -Ot -o {o}'.format(r=ref,g=gff,o=out)
    print (cmd)
    tmp = subprocess.check_output(cmd,shell=True)
    
csq_call()

bcftools csq -f /home/damien/.config/snpgenie/genome/Mbovis_AF212297.fa -g ../snpgenie/data/Mbovis_csq_format.gff mapped/filtered.vcf.gz -Ot -o mapped/csq.tsv


In [ ]:
reload(app)
def read_csq_file(filename):
    cols = ['1','filename','2','chrom','start','snp_type','gene','locus_tag','strand','feature_type','aa','nuc',]
    csqdf = pd.read_csv(filename,sep='[|\t]',comment='#',names=cols, engine='python')
    return csqdf
csq = read_csq_file('csqout.tsv')
csq

In [34]:
def vcf_to_dataframe(vcf_file, quality=30):
    """Convert vcf to dataframe"""

    import vcf
    ext = os.path.splitext(vcf_file)[1]    
    if ext == '.gz':
        file = gzopen(vcf_file, "rt")
    else:
        file = open(vcf_file)
    vcf_reader = vcf.Reader(file,'r')    
    res=[]
    cols = ['sample','REF','ALT','mut','chrom','var_type','sub_type','start','end','QUAL','DP']
    i=0
    for rec in vcf_reader:
        #if i>50:
        #    break
        S = {sample.sample: sample.gt_bases for sample in rec.samples}
        #print (S)
        x = [rec.CHROM, rec.var_type, rec.var_subtype, rec.start, rec.end, rec.QUAL, rec.INFO['DP']]
        for sample in rec.samples:
            if sample.gt_bases==None:
                mut=''
            elif rec.REF != sample.gt_bases:
                mut=str(rec.end)+rec.REF+'>'+sample.gt_bases
            else:
                mut = rec.REF
            row = [sample.sample, rec.REF, sample.gt_bases, mut] + x
            res.append(row)       
        #i+=1
    res = pd.DataFrame(res,columns=cols)    
    return res


In [35]:
vdf = vcf_to_dataframe('../mapped/filtered.vcf.gz')
vdf

,sample,REF,ALT,mut,chrom,var_type,sub_type,start,end,QUAL,DP
0,mapped/SRR10482933.bam,G,G,G,NC_002945.4,snp,ts,44,45,999.0,1258
1,mapped/SRR10482932.bam,G,G,G,NC_002945.4,snp,ts,44,45,999.0,1258
2,mapped/SRR1791710.bam,G,A,45G>A,NC_002945.4,snp,ts,44,45,999.0,1258
3,mapped/SRR8063663.bam,G,A,45G>A,NC_002945.4,snp,ts,44,45,999.0,1258
4,mapped/SRR8063661.bam,G,A,45G>A,NC_002945.4,snp,ts,44,45,999.0,1258
...,...,...,...,...,...,...,...,...,...,...,...
98371,mapped/SRR5486092.bam,G,None,,NC_002945.4,snp,ts,4348833,4348834,213.0,1311
98372,mapped/SRR5486090.bam,G,G,G,NC_002945.4,snp,ts,4348833,4348834,213.0,1311
98373,mapped/SRR5486087.bam,G,G,G,NC_002945.4,snp,ts,4348833,4348834,213.0,1311
98374,mapped/SRR8063660.bam,G,G,G,NC_002945.4,snp,ts,4348833,4348834,213.0,1311


In [31]:
def get_snp_matrix(df):
    """SNP matrix from multi sample vcf dataframe"""
    
    df = df.drop_duplicates(['mut','sample'])
    x = df.set_index(['mut','sample'])['start'].unstack('sample')
    x[x.notna()] = 1
    x = x.fillna(0)
    return x

sample,mapped/3_S57_L001.bam,mapped/6_S7_L001.bam,mapped/SRR10482932.bam,mapped/SRR10482933.bam,mapped/SRR1791710.bam,mapped/SRR1791711.bam,mapped/SRR1791712.bam,mapped/SRR5486071.bam,mapped/SRR5486072.bam,mapped/SRR5486087.bam,...,mapped/SRR5486092.bam,mapped/SRR8063651.bam,mapped/SRR8063652.bam,mapped/SRR8063654.bam,mapped/SRR8063660.bam,mapped/SRR8063661.bam,mapped/SRR8063662.bam,mapped/SRR8063663.bam,mapped/SRR8063665.bam,mapped/SRR8063666.bam
mut,,,,,,,,,,,,,,,,,,,,,
,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1000231A>G,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000631C>T,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10031C>T,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004312TGGGGGG>TGGGGGGG,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004501C>T,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004701C>T,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1005199G>A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1009406C>T,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Make fasta alignment from filtered VCF sites with multiple samples

https://github.com/JosephCrispell/GeneralTools/blob/master/ProcessingPipeline/CreateFastaWithReferenceFromFiltered_28-06-17.pl

"The allele frequencies at each position in the aligned (against reference) sequence from each isolate were examined. For a haploid organism these frequencies are expected to be either 0 or 1, with some random variation expected from sequencing errors (Sobkowiak et al., 2018). A heterozygous site was defined as one where the allele frequencies were >0.05 and <0.95. Four cattle-derived sequences that had more than 150 heterozygous sites, and allele frequencies that were clustered and non-random (data not shown), were removed. In addition, 26 badger-derived and 16 cattle-derived M. bovis sequences were removed because of suspected errors in the metadata (Appendix 1: Investigating isolate metadata discrepancies).

For the sequences from the remaining isolates (204 badger- and 169 cattle-derived isolates), alleles were called at each variant position if they had mapping quality ≥30, high-quality base depth ≥4 (applied to reverse and forward reads separately), read depth ≥30, and allele support ≥0.95. For any site that failed these criteria, if the allele called had been observed in a different isolate that had passed, a second round of filtering was conducted using a high-quality base depth of 5 (total across forward and reverse reads) and the same allele support. As recombination is thought to be extremely rare for mycobacteria (Namouchi et al., 2012), variants in close proximity could indicate a region that is difficult to sequence or under high selection. To avoid calling variants in these regions, variant positions within 10 bp of one another were removed. Following filtering, sequences from 11 badger and 10 cattle isolates that had insufficient coverage (<95%) of the variant positions were removed. Once the alignment was generated, sites with a consistency index less than 1, generally considered homoplasies (Farris, 1989), were removed (n = 4, of 14,991 sites) using HomoplasyFinder (v0.0.0.9; Crispell et al., 2019; RRID: SCR_017300). All the scripts necessary for the processing of the WGS data are freely available online."

In [252]:
def fasta_alignment_from_vcf(vcf_file, callback=None, uninformative=False):
    """Get core snp site calls as sequences from all samples in a vcf file"""

    import vcf
    from collections import defaultdict
    vcf_reader = vcf.Reader(open(vcf_file, 'rb'))
    #print (vcf_reader.samples)
    def default():
        return []
    result = defaultdict(default)
    sites = []
    result['ref'] = []
    for record in vcf_reader:
        S = {sample.sample: sample.gt_bases for sample in record.samples}
        #if any missing samples at the site we don't add
        if None in S.values():
            #print (S)
            continue
        if uninformative == False:
            u = set(S.values())
            if len(u) == 1:
                continue
        result['ref'].append(record.REF)
        for name in S:
            result[name].append(S[name])
        sites.append(record.POS)

    #print (result)
    print ('found %s sites' %len(sites))
    recs = []
    for sample in result:
        seq = ''.join(result[sample])
        seqrec = SeqRecord(Seq(seq),id=sample)
        recs.append(seqrec)
        #print (len(seqrec))

    smat = pd.DataFrame(result)
    smat.index = sites
    smat.index.rename('pos', inplace=True)    
    return recs, smat

#seqrecs, smat = fasta_alignment_from_vcf('mapped/filtered.vcf.gz')
seqrecs, smat = fasta_alignment_from_vcf('/storage/btbgenie/wicklow_results/filtered.vcf.gz')
SeqIO.write(seqrecs, 'core.fa', 'fasta')

found 531 sites


47

## filters

In [120]:
import tempfile
def site_proximity_filter(vcf_file, dist=10, outdir='mapped'):
    """Remove any pairs of sites within dist of each other"""

    import vcf
    vcf_reader = vcf.Reader(open(vcf_file, 'rb'))
    sites = [record.POS for record in vcf_reader]
    #print (sites)
    found = []
    for i in range(len(sites)-1):
        if sites[i+1] - sites[i] <= dist:
            #print (sites[i], sites[i+1],'close')
            found.extend([sites[i], sites[i+1]])
    #print (found)
    new = sorted(list(set(sites) - set(found)))
    print ('proximity filter found %s/%s sites' %(len(new),len(sites)))
    tempdir = tempfile.gettempdir()
    out = os.path.join(outdir,'temp.vcf')
    vcf_reader = vcf.Reader(open(vcf_file, 'rb'))
    vcf_writer = vcf.Writer(open(out, 'w'), vcf_reader)
    for record in vcf_reader:
        if record.POS in new:
            #print (record)
            vcf_writer.write_record(record)
    vcf_writer.close()
    #overwrite input vcf
    bcftoolscmd = tools.get_cmd('bcftools')
    cmd = 'bcftools view {o} -O z -o {gz}'.format(o=out,gz=vcf_file)
    tmp = subprocess.check_output(cmd,shell=True)
    os.remove(out)
    return

site_proximity_filter('mapped/filtered.vcf.gz')

proximity filter found 1550/1634 sites


## mask locations for mbovis

In [ ]:
#make mask bed file for mbovis
g=tools.genbank_to_dataframe('../snpgenie/data/Mbovis_AF212297.gb')
g['gene']=g.gene.fillna('')
g=g[(g.gene.str.contains('PE_PGRS') | g.gene.str.contains('PPE')) & (g.feat_type=='CDS')]
for i,r in g.iterrows():
    print ('LT708304.1','\t', r.start, '\t', r.end, r.gene, r.locus_tag)

In [ ]:
def mask_filter(vcf_file, mask_file):
    """Remove any masked sites using a bed file, overwrites input"""

    print('using mask bed file', mask_file)
    mask = pd.read_csv(mask_file,sep='\t',names=['chrom','start','end'])    
    #print (mask)
    def do_mask(x,i):     
        #print (x.start, x.end, i)
        if (x.start<=i) & (x.end>=i):            
            return 1
    import vcf
    vcf_reader = vcf.Reader(open(vcf_file, 'rb'))
    sites = [record.POS for record in vcf_reader]
    found = []
    for i in sites:       
        m = mask.apply( lambda x: do_mask(x,i),1)        
        m = m[m==1]   
        if len(m)>0:    
            print (i)
            found.append(i)
    print('found %s sites in masked regions' %len(found))
    new = sorted(list(set(sites) - set(found)))
    
    tempdir = tempfile.gettempdir()
    out = os.path.join(tempdir,'temp.vcf')
    vcf_reader = vcf.Reader(open(vcf_file, 'rb'))
    vcf_writer = vcf.Writer(open(out, 'w'), vcf_reader)
    for record in vcf_reader:
        if record.POS in new:
            #print (record)
            vcf_writer.write_record(record)
    vcf_writer.close()
    #overwrite input vcf
    bcftoolscmd = tools.get_cmd('bcftools')
    cmd = 'bcftools view {o} -O z -o {gz}'.format(o=out,gz=vcf_file)
    tmp = subprocess.check_output(cmd,shell=True)
    return

mask_filter('mapped/filtered.vcf.gz','../snpgenie/data/Mbovis_AF212297_mask.bed')


## uninformative sites

probably not needed as we can use `--min-ac 1:minor`  in bcftools view?

In [237]:
def uninformative_sites_filter(vcf_file):
    """Ree"""

    import vcf
    vcf_reader = vcf.Reader(open(vcf_file, 'rb'))
    i=0; u=0
    for record in vcf_reader:
        ref=record.REF
        S = set([sample.gt_bases for sample in record.samples])
        #print (record.POS,ref,S)
        if (len(S)==1):
            #print (record.POS,ref,S)
            u+=1
        i+=1
    print (i,u)    
    
#u_filter('mapped/filtered.vcf.gz')
#u_filter('../test_results/filtered.vcf.gz')
uninformative_sites_filter('/storage/btbgenie/wicklow_results/filtered.vcf.gz')

629 78


In [210]:
def generic_multiprocess(inputs):
    #split sites into chunks and run in parallel for long jobs
    
    import multiprocessing as mp
    blocks = np.array_split(np.array(inputs),threads)
    result=[]
    funclist = []
    pool = mp.Pool(threads)
    for subset in blocks:            
        f = pool.apply_async(worker_func, [subset])      
        funclist.append(f)            
    try:
        for f in funclist:
            out = f.get(timeout=None)               
            if len(out)>0:
                result.append(out)
    except KeyboardInterrupt:
        print ('process interrupted')
        pool.terminate()
        sys.exit(0)

    pool.close()
    pool.join()    
    return result

### Phylogeny from MSA

In [ ]:
def run_RAXML(infile, name='variants', threads=8):
    """Run Raxml pthreads"""

    bootstraps = 10
    model = 'GTRCAT'
    s1 = random.randint(0,1e8)
    s2 = random.randint(0,1e8)
    
    files = glob.glob('RAxML_*')
    for f in files:
        os.remove(f)
    cmd = 'raxmlHPC-PTHREADS -f a -N {nb} -T {t} -m {m} -V -p {s1} -x {s2} -n {n} -s {i}'.format(t=threads,nb=bootstraps,n=name,i=infile,s1=s1,s2=s2,m=model)
    print (cmd)
    tmp = subprocess.check_output(cmd, shell=True)
    return
    
run_RAXML('variants.fa')

In [ ]:
def create_tree(filename, labelmap=None):
    """Draw a tree """
    
    from ete3 import Tree, NodeStyle, TreeStyle
    t = Tree(filename)
    t.set_outgroup('ref')    
    if labelmap != None:
        trees.set_tiplabels(t,labelmap)
    trees.format_nodes(t)
    ts = TreeStyle()
    ts.scale=1000
    t.render("%%inline", tree_style=ts)
    t.render("tree.png", tree_style=ts)
    return t

sra['filename'] = sra.Run.apply(lambda x: 'mapped/'+x+'.bam')
print (sra[:3])
labelmap = dict(zip(sra.filename,sra.geo_loc_name_country))
#print (labelmap)
t = create_tree('RAxML_bipartitions.variants', labelmap)
t.render("%%inline")

In [ ]:
from Bio import Phylo
tree=Phylo.read('RAxML_bestTree.variants','newick')
Phylo.draw(tree)

## csq gff format

In [ ]:
tools.gff_bcftools_format('../snpgenie/data/Mbovis_AF212297.gb', '../Mbovis_csq_new.gff')

In [ ]:
gff=tools.gff_to_features('../pathogenie/data/Mbovis_AF212297.gff')
g=tools.features_to_dataframe(gff)
tools.features_summary(g)

In [ ]:
reload(plotting)
f,ax=plt.subplots(1,1,figsize=(12,3))
st=time.time()
plotting.plot_bam_alignment('../test_results/mapped/3_S57_L001.bam', 'LT708304.1', 2000, 3000, yend=100, rect_height=3, ax=ax)
print (time.time()-st)

In [ ]:
def plot_features(rec, ax, rows=3, xstart=0, xend=30000):
    
    h=1
    df = tools.records_to_dataframe([rec])    
    df = df[(df.feat_type!='region') & (df['feat_type']!='source')]
    df = df[(df.start>xstart) & (df.end<xend)]
    df['length'] = df.end-df.start
    y = list(range(1,rows)) * len(df)
    df['y'] = y[:len(df)]
    df['color'] = 'blue'
    df = df.fillna('')
    #print (df)  
    
    from matplotlib.collections import PatchCollection
    import matplotlib.patches as mpatches

    patches=[]
    for i,r in df.iterrows():        
        #rect = plt.Rectangle((r.start, r.y), r.length, h/2,
        #                        alpha=.5, linewidth=.5,
        #                        edgecolor='black', facecolor=r.color)
        if r.strand == 1:
            x = r.start
            dx = r.length
        else:
            x = r.end
            dx = -r.length
        arrow = mpatches.Arrow(x, r.y, dx, 0, alpha=.7, width=.3, 
                               edgecolor='black')
        txt = ax.text(r.start, r.y-h/2, r.gene, size=16)
        patches.append(arrow)  
        
    ax.add_collection(PatchCollection(patches, match_original=True))  
    ax.set_xlim(xstart, xend)
    ax.set_ylim(.4,rows-.5)
    plt.yticks([])
    plt.tight_layout()

    def onclick(event):
        print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
              ('double' if event.dblclick else 'single', event.button,
               event.x, event.y, event.xdata, event.ydata))
        ax.text(event.x, event.y, 'HI!')
        ax.figure.canvas.draw()
    #cid = ax.figure.canvas.mpl_connect('button_press_event', onclick)
    return

reload(tools)
recs = tools.gff_to_records(app.mbovis_gff)
rec=recs[0]
f,ax=plt.subplots(1,1,figsize=(15,1.5))
plot_features(rec, ax=ax, xstart=26000,xend=35000)

## replace pysam

In [ ]:
def get_chrom(bam_file):
    """Get first sequence name in a bam file"""

    import pysam
    samfile = pysam.AlignmentFile(bam_file, "r")
    iter=samfile.fetch(start=0,end=10)
    for read in iter:
        if read.reference_name:
            return read.reference_name
        
get_chrom('mapped/3_S57_L001.bam')

In [4]:
import bamnostic as bs
bam = bs.AlignmentFile('mapped/3_S57_L001.bam', 'rb')

In [6]:
chrom=bam.header['SQ'][0]['SN']
chrom

'LT708304.1'

In [ ]:
for i, read in enumerate(bam.fetch(chrom, 1, 10)):
    print(read)

In [ ]:
def get_coverage(bam_file, chr, start, end):
    """Get coverage from bam file at specified region"""
    
    import bamnostic as bs
    if bam_file is None or not os.path.exists(bam_file):
        return
    bam = bs.AlignmentFile(bam_file, 'rb')
    for i, read in enumerate(bam.fetch(chrom, 1, 10)):
        print(read)
    return

get_coverage('mapped/3_S57_L001.bam', chrom, 1, 100)

In [ ]:
def samtools_flagstats(filename):
    """Parse samtools flagstat output into dictionary"""
    
    cmd = 'samtools flagstat %s' %filename
    tmp = subprocess.check_output(cmd, shell=True, universal_newlines=True)
    x = tmp.split('\n')    
    x = [int(i.split('+')[0]) for i in x[:-1]]
    #print (x)
    cols = ['total','secondary','supplementary','duplicates','mapped',
            'paired','read1','read2','properly paired','with itself','singletons']
    d = {}
    for c,v in zip(cols,x):
        d[c] = v
    return d

samtools_flagstats('mapped/6_S7_L001.bam')

In [ ]:
def samtools_tview(bam_file, chrom, pos, ref='', display='T'):
    """View bam alignment with samtools"""
    
    cmd = 'COLUMNS=150 samtools tview {b} -p {c}:{p} -d {d} {r}'.format(b=bam_file,c=chrom,p=pos,d=display,r=ref)
    #print (cmd)
    tmp = subprocess.check_output(cmd, shell=True, universal_newlines=True)    
    return tmp

t=samtools_tview('mapped/6_S7_L001.bam', 'LT708304.1', 1900, app.mbovis_genome)
print (t)

In [ ]:
def samtools_depth(bam_file, chrom, start, end):
    """Get depth from bam file"""

    cmd = 'samtools depth -r {c}:{s}-{e} {b}'.format(b=bam_file,c=chrom,s=start,e=end)
    tmp=subprocess.check_output(cmd, shell=True, universal_newlines=True)
    from io import StringIO
    c = pd.read_csv(StringIO(tmp),sep='\t',names=['chr','pos','depth'])
    return c
    
t=samtools_depth('mapped/6_S7_L001.bam', 'LT708304.1', 1900, 2000)
t

In [ ]:
reload(tools)
tools.plot_fastq_qualities(files[3],limit=100000)
tools.plot_fastq_qualities(files[3],limit=40000)